# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ch0125\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ch0125\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ch0125\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponses.db')
df = pd.read_sql_table('Response', engine)

In [3]:
# check imported table
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [4]:
# specify dependent and independent variables
X = df.loc[:, ['message']]
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    # remove puntuaton and special chars
    text = re.sub(r'[^\w]', ' ', text)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
        
    # lemmatize, normalize case, and remove leading/trailing white space
    stop_words = stopwords.words('english')
    final_tokens = [lemmatizer.lemmatize(token).strip().lower() 
                        for token in tokens 
                            if token not in stop_words and len(token) > 2]
    return final_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('count', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('multi_clf', MultiOutputClassifier(RandomForestClassifier())),
])   

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
# create train test split
X_train, X_test, y_train, y_test = train_test_split(X.message.values, y, test_size=.25, random_state=42)

# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('count', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
# Make prediction
pred = pipeline.predict(X_test)

In [49]:
print(classification_report(y_test.values[:, 2], pred[:, 2]))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6516
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6545



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [51]:
f1_score(y_test.values[:, 2], pred[:, 2], average='macro')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.49888982466886145

In [28]:
from sklearn.metrics import classification_report

# Print report
for i in range(36):
    print(classification_report(y_test.values[:, i], pred[:, i]))

             precision    recall  f1-score   support

          0       0.65      0.47      0.54      1557
          1       0.84      0.92      0.88      4941
          2       0.40      0.49      0.44        47

avg / total       0.79      0.81      0.80      6545

             precision    recall  f1-score   support

          0       0.89      0.97      0.93      5393
          1       0.78      0.45      0.57      1152

avg / total       0.87      0.88      0.87      6545

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6516
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6545

             precision    recall  f1-score   support

          0       0.75      0.85      0.80      3801
          1       0.74      0.61      0.67      2744

avg / total       0.75      0.75      0.74      6545

             precision    recall  f1-score   support

          0       0.92      0.99 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [42]:
from sklearn.metrics import f1_score

scores = []
for i in range(36):
    score = f1_score(y_test.values[:, i], pred[:, i], average='macro')
    scores.append(score)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [55]:
print('Unweighted f1-score averaged over all columns: %.2f' % (sum(scores)/len(scores)))

Unweighted f1-score averaged over all columns: 0.61


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.